# Homework: Vector Search


## Q1. Getting the embeddings model

In [26]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

c:\Users\User\Documents\llm-zoomcamp-dbeta95\llm-zoomcampt-3.10\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
user_question = "I just discovered the course. Can I still join it?"

In [28]:
res_vector = embedding_model.encode(user_question)

In [29]:
res_vector[0]

0.07822266

In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [12]:
documents = [doc for doc in documents if doc['course']=='machine-learning-zoomcamp']

In [14]:
len(documents)

375

## Q2. Creating the embeddings

In [17]:
embeddings = []
for doc in documents:
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(embedding_model.encode(doc['text']))

In [19]:
import numpy as np

X = np.array(embeddings)

In [21]:
X.shape

(375, 768)

## Q3. Search



In [30]:
v = embedding_model.encode(user_question)

In [31]:
scores = X.dot(v)

In [32]:
scores.argmax()

15

In [33]:
scores[15]

0.4749719

## Vector search

In [37]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'The course is available in the self-paced mode too, so you can go through the materials at any time. But if you want to do it as a cohort with other students, the next iterations will happen in September 2023, September 2024 (and potentially other Septembers as well).',
  'section': 'General course-related questions',
  'question': 'When does the next iteration start?',
  'course': 'machine-learning-zoomcamp',
  'id': '636f55d5'},
 {'text': "The process is automated now, so you should receive the email eventually. If you haven’t, check your promotions tab in Gmail as well as spam.\nIf you unsubscribed from our newsletter, you won't get course related updates too.\nBut don't worry, it’s not a problem. To make sure you don’t miss anything, join the #course-ml-zoomcamp channel in Slack and our telegram channel with announcements. This is enough to follow the course.",
  'section': 'General course-related questions',
  'question': "I filled the form, but haven't received a confi

In [46]:
def search_fun(query):
    v = embedding_model.encode(query)
    return search_engine.search(v, num_results=5)

In [38]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [41]:
ground_truth

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Does this course have a GitHub repository for the sign-up link?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'How can I structure my questions and answers for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Are the course videos live or pre-recorded?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'When can I start watching the course videos?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'Are the live office hours sessions recorded?',
  'cours

In [39]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [48]:
from tqdm import tqdm
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [50]:
evaluate(ground_truth, lambda q: search_fun(q['question']))

100%|██████████| 1830/1830 [00:43<00:00, 41.76it/s]


{'hit_rate': 0.8803278688524591}

In [53]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [58]:
for i, doc in enumerate(documents):
    doc["text_vector"] = embeddings[i]
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [55]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [68]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [64]:
reponse = elastic_search(user_question, 'machine-learning-zoomcamp')

In [67]:
reponse['hits']

{'total': {'value': 351, 'relation': 'eq'},
 'max_score': 76.8762,
 'hits': [{'_index': 'course-questions',
   '_id': '4PmrxJABfD3OXpZzopFn',
   '_score': 76.8762,
   '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
    'section': 'General course-related questions',
    'question': 'The course has already started. Can I still join it?',
    'course': 'machine-learning-zoomcamp',
    'id': 'ee58a693',
    'text_vector': [0.09511076658964157,
     -0.04694204777479172,
     -0.006974613294005394,
     -0.0070084440521895885,
     0.05774153769016266,
     -0.046521518379449844,
     0.005525636486709118,
     0.013560931198298931,
     0.00738000

In [69]:

evaluate(ground_truth, lambda q: elastic_search(q['question'], 'machine-learning-zoomcamp'))

100%|██████████| 1830/1830 [01:33<00:00, 19.57it/s]


{'hit_rate': 0.7530054644808744}

embbedings